# Selection of PET **TAU** images to process

In [1]:
import os
import pandas as pd 
import numpy as np
from tableone import TableOne

In [2]:
dir = '/scratch/caroline/papers/ongoing/project00/ADNI_analysis/data'
tau_ = pd.read_csv(os.path.join(dir, 'UCBERKELEY_TAUPVC_6MM_03Sep2024.csv'))
tau_ = tau_.rename(columns={'PTID': 'Individual', 'SCANDATE':'tau_date'})
tau = tau_.copy()
img = pd.read_csv('/scratch/caroline/papers/ongoing/project00/BA_predictions_2025/data/adni/adni_nifti_20250522.csv',usecols=['GUID','Individual','Timepoint', 'Scan date', 'Alt ImageID'])
img = img.rename(columns={'Scan date':'mri_date'})
img['mri_date'] = pd.to_datetime(img['mri_date'])
orig = pd.read_csv(os.path.join(dir, 'adni_original/PTDEMOG_05Jun2025.csv'), 
                   usecols=['PTID','RID', 'VISDATE', 'PTGENDER', 'PTDOB', 'PTEDUCAT'])
orig = orig.rename(columns={'PTID':'Individual'})

In [3]:
# put date to datetime
tau['tau_date'] = pd.to_datetime(tau['tau_date'])
# start selecting to merge
tau = tau[['LONIUID', 'VISCODE', 'Individual', 'tau_date']]

# CDR

In [4]:
#import data
cdr = pd.read_csv(os.path.join(dir, 'adni_original/CDR_05Jun2025.csv'), usecols=['PTID', 'VISDATE', 'CDRSB', 'VISCODE2', 'VISCODE'])
cdr = cdr.rename(columns={'PTID':'Individual'})

In [5]:
cdr

,Individual,VISCODE,VISCODE2,VISDATE,CDRSB
0,011_S_0002,sc,sc,2005-08-17,0.0
1,011_S_0003,sc,sc,2005-08-18,4.5
2,022_S_0004,sc,sc,2005-08-18,1.0
3,011_S_0005,sc,sc,2005-08-23,0.0
4,022_S_0007,sc,sc,2005-08-25,6.0
...,...,...,...,...,...
14309,131_S_10815,4_sc,sc,2025-06-02,1.5
14310,126_S_10722,4_sc,sc,2025-06-03,0.0
14311,137_S_6903,4_m12,m42,2025-05-05,0.5
14312,010_S_6748,4_init,m72,2025-06-03,1.0


In [6]:
# merge dataset
cdr_tau = pd.merge(tau, cdr, on=['Individual','VISCODE'],  how='left')
cdr_tau = cdr_tau.drop_duplicates()
cdr_tau['VISDATE'] = pd.to_datetime(cdr_tau['VISDATE'], format='%Y-%m-%d')
# difference in months
cdr_tau['date_diff'] = abs((cdr_tau['VISDATE'] - cdr_tau['tau_date']).dt.days / 30)
#Keep only images that were taken within 3 months (MRI-TAU)
adni1 = cdr_tau[(cdr_tau['date_diff'] <= 7)].copy()
print(adni1.Individual.nunique())
# 3 months = 504, 4 months = 519, 5 months = 530, 6 months = 533, , 7 months = 539, 8 months = 540, 12 monrhs = 542

539


# MMSE


In [7]:
# import data
mmse = pd.read_csv(os.path.join(dir, 'adni_original/MMSE_05Jun2025.csv'), usecols=['PTID', 'VISDATE', 'MMSCORE', 'VISCODE2', 'VISCODE'])
mmse = mmse.rename(columns={'PTID':'Individual'})

In [8]:
mmse

,Individual,VISCODE,VISCODE2,VISDATE,MMSCORE
0,011_S_0002,sc,sc,2005-08-17,28.0
1,022_S_0001,f,f,2005-08-18,28.0
2,011_S_0003,sc,sc,2005-08-18,20.0
3,022_S_0004,sc,sc,2005-08-18,27.0
4,011_S_0005,sc,sc,2005-08-23,29.0
...,...,...,...,...,...
14331,137_S_6903,4_m12,m42,2025-05-05,29.0
14332,010_S_6748,4_init,m72,2025-06-03,29.0
14333,123_S_10816,4_sc,sc,2025-06-03,28.0
14334,127_S_6436,4_init,m84,2025-05-20,30.0


In [9]:
# merge dataset
mmse_tau = pd.merge(tau, mmse, on=['Individual','VISCODE'],  how='left')
mmse_tau = mmse_tau.drop_duplicates()
mmse_tau['VISDATE'] = pd.to_datetime(mmse_tau['VISDATE'], format='%Y-%m-%d')
# difference in months
mmse_tau['date_diff'] = abs((mmse_tau['VISDATE'] - mmse_tau['tau_date']).dt.days / 30)

In [12]:
#Keep only images that were taken within 3 months (MRI-TAU)
adni2 = mmse_tau[(mmse_tau['date_diff'] <= 7)].copy()
print(adni2.Individual.nunique())
# 3 months = 510, 4 months = 527, 5 months = 538, 6 months = 541, , 7 months = 547, 8 months = 548, 12 monrhs = 550

547


# MoCA

#https://www.smchealth.org/sites/main/files/file-attachments/moca-instructions-english_2010.pdf
#https://www.smchealth.org/sites/main/files/file-attachments/moca-instructions-english_2010.pdf

In [13]:
# import data
moca = pd.read_csv(os.path.join(dir, 'adni_original/MOCA_05Jun2025.csv'))#, usecols=['PTID', 'VISDATE', 'MMSCORE', 'VISCODE2', 'VISCODE'])
moca = moca.drop(columns=[ 'ID', 'SITEID', 'USERDATE', 'USERDATE2', 'DD_CRF_VERSION_LABEL', 'LANGUAGE_CODE', 'HAS_QC_ERROR','update_stamp'])
moca = moca.rename(columns={'PTID':'Individual'})

In [14]:
moca

,PHASE,Individual,RID,VISCODE,VISCODE2,VISDATE,TRAILS,CUBE,CLOCKCON,CLOCKNO,...,DELW4,DELW5,DATE,MONTH,YEAR,DAY,PLACE,CITY,MOCA,SOURCE
0,ADNIGO,035_S_0292,292,m48,m48,2010-04-21,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN
1,ADNIGO,016_S_1326,1326,m36,m36,2010-05-12,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,NaN
2,ADNIGO,023_S_0376,376,m48,m48,2010-05-14,1.0,1.0,1.0,1.0,...,0.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN
3,ADNIGO,016_S_1117,1117,m36,m36,2010-06-24,0.0,0.0,0.0,1.0,...,0.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0,NaN,NaN
4,ADNIGO,031_S_0618,618,m48,m48,2010-06-25,1.0,1.0,1.0,0.0,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8612,ADNI4,022_S_10361,10361,4_bl,bl,2025-03-27,1.0,1.0,1.0,1.0,...,0.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,27.0,1.0
8613,ADNI4,114_S_10675,10675,4_bl,bl,2025-05-28,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,25.0,1.0
8614,ADNI4,137_S_6903,6903,4_m12,m42,2025-05-05,1.0,1.0,1.0,1.0,...,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,28.0,1.0
8615,ADNI4,010_S_6748,6748,4_init,m72,2025-06-03,1.0,0.0,1.0,1.0,...,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,22.0,1.0


In [15]:
moca_df = moca.copy()
#ADjust variables
moca_df = moca_df.drop(columns=['IMMT1W1', 'IMMT1W2', 'IMMT1W3', 'IMMT1W4', 'IMMT1W5', 'IMMT2W1', 'IMMT2W2', 'IMMT2W3', 'IMMT2W4', 'IMMT2W5' ])

# Letters = 0 or 1 error gives 1 point, else is 0
moca_df['letters_bi'] = np.where(moca_df['LETTERS'].isin([0, 1]), 1, 0)
moca_df = moca_df.drop(columns=['LETTERS'])

#FLUENCY
# if more than 0 < 11 < 1
moca_df['fluency'] = (moca_df['FFLUENCY'] >= 11).astype(int)
moca_df = moca_df.drop(columns=['FFLUENCY'])

# SERIAL 
#Give no (0) points for no correct subtractions, 1 point for one correction subtraction, 2 points for two-to-three correct subtractions, and 3 points if the participant successfully makes four or five correct subtractions
serial_cols = [col for col in moca_df.columns if col.startswith('SERIAL')]
moca_df['serial_sum'] = moca_df[serial_cols].sum(axis=1)
moca_df['serial_sum'] = np.where(moca_df['serial_sum'].isin([4, 5]), 3, 0)
moca_df = moca_df.drop(columns=['SERIAL1', 'SERIAL2', 'SERIAL3', 'SERIAL4', 'SERIAL5'])

#DEL
# 1=Correct with No Cue; 2=Correct with Category Cue; 3=Correct with Mult. Choice Cue; 0=Incorrect
# 1 point only with correct with no cue
moca_df['delw1'] = (moca_df['DELW1'] == 1).astype(int)
moca_df['delw2'] = (moca_df['DELW2'] == 1).astype(int)
moca_df['delw3'] = (moca_df['DELW3'] == 1).astype(int)
moca_df['delw4'] = (moca_df['DELW4'] == 1).astype(int)
moca_df['delw5'] = (moca_df['DELW5'] == 1).astype(int)
moca_df = moca_df.drop(columns=['DELW1', 'DELW2', 'DELW3', 'DELW4', 'DELW5'])

moca_df['moca_score'] = moca_df.iloc[:,8:].sum(axis=1)


In [16]:
# merge dataset
moca_tau = pd.merge(tau, moca, on=['Individual','VISCODE'],  how='left')
moca_tau = moca_tau.drop_duplicates()
moca_tau['VISDATE'] = pd.to_datetime(moca_tau['VISDATE'], format='%Y-%m-%d')
# difference in months
moca_tau['date_diff'] = abs((moca_tau['VISDATE'] - moca_tau['tau_date']).dt.days / 30)

In [ ]:
#Keep only images that were taken within 3 months (MRI-TAU)
adni3 = moca_tau[(moca_tau['date_diff'] <= 7)].copy()
print(adni3.Individual.nunique())
# 3 months = 843, 4 months = 867, 5 months = 876, 6 months = 885, 7 months = 889, 8 months = 892, 12 monrhs = 895

895


# DEMO

In [40]:
# import data
demo = pd.read_csv(os.path.join(dir, 'adni_original/PTDEMOG_05Jun2025.csv'), usecols=['PTID', 'PTGENDER', 'PTDOB','PTDOBYY','PTEDUCAT'])
demo = demo.rename(columns={'PTID':'Individual'})

In [41]:
demo

,Individual,PTGENDER,PTDOB,PTDOBYY,PTEDUCAT
0,011_S_0002,1.0,04/1931,1931.0,16.0
1,022_S_0001,2.0,12/1944,1944.0,18.0
2,011_S_0003,1.0,05/1924,1924.0,18.0
3,022_S_0004,1.0,01/1938,1938.0,10.0
4,011_S_0005,1.0,12/1931,1931.0,16.0
...,...,...,...,...,...
6041,305_S_10746,2.0,03/1967,1967.0,16.0
6042,082_S_10809,2.0,08/1945,1945.0,15.0
6043,002_S_10814,1.0,03/1948,1948.0,18.0
6044,052_S_6412,1.0,07/1951,1951.0,16.0


In [44]:
# merge dataset
demo_tau = pd.merge(tau, demo, on=['Individual'],  how='left')
demo_tau = demo_tau.drop_duplicates()
demo_tau = demo_tau.dropna()

#Keep only images that were taken within 3 months (MRI-TAU)
adni3 = demo_tau.copy()
print(adni3.Individual.nunique())

907


In [62]:
# import data
diagnosis = pd.read_csv(os.path.join(dir, 'adni_original/DXSUM_05Jun2025.csv'), usecols=['PTID', 'RID', 'VISCODE', 'VISCODE2', 'EXAMDATE', 'DIAGNOSIS','DXPARK'])#,'DXCURREN', 'DXCONV','DXCHANGE'])
diagnosis = diagnosis.rename(columns={'PTID':'Individual'})

In [56]:
diagnosis.columns

Index(['PHASE', 'Individual', 'RID', 'VISCODE', 'VISCODE2', 'EXAMDATE',
       'DIAGNOSIS', 'DXNORM', 'DXNODEP', 'DXMCI', 'DXMDES', 'DXMPTR1',
       'DXMPTR2', 'DXMPTR3', 'DXMPTR4', 'DXMPTR5', 'DXMPTR6', 'DXMDUE',
       'DXMOTHET', 'DXDSEV', 'DXDDUE', 'DXAD', 'DXAPP', 'DXAPROB', 'DXAPOSS',
       'DXPARK', 'DXPDES', 'DXPCOG', 'DXPATYP', 'DXDEP', 'DXOTHDEM', 'DXODES',
       'DXCONFID', 'ID', 'SITEID', 'USERDATE', 'USERDATE2',
       'DD_CRF_VERSION_LABEL', 'LANGUAGE_CODE', 'HAS_QC_ERROR',
       'update_stamp'],
      dtype='object')

In [ ]:
# import data
conv = pd.read_csv(os.path.join(dir, 'adni_original/CONVERSIONS_05Jun2025.csv'), usecols=['RID', 'VISCODE', 'TYPE'])
conv


,RID,VISCODE,TYPE
0,41,m18,MCI to AD
1,77,m12,MCI to AD
2,128,m18,MCI to AD
3,205,m06,MCI to NL
4,223,m06,NL to MCI
5,243,m06,MCI to AD
6,249,m12,MCI to AD
7,269,m06,MCI to AD
8,344,m12,MCI to AD
9,476,m12,MCI to AD


In [60]:
# import data
X = pd.read_csv(os.path.join(dir, 'adni_original/ADNIMERGE_05Jun2025.csv'))
X.PTID.nunique()


/tmp/ipykernel_2976116/3156411768.py:2: DtypeWarning: Columns (19,20,21,50,51,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  X = pd.read_csv(os.path.join(dir, 'adni_original/ADNIMERGE_05Jun2025.csv'))


2430

# image

In [ ]:
#merge date based on individuals that have tau
df_id = pd.merge(tau, img, on='Individual',  how='right')
# difference in months
df_id['date_diff'] = abs((df_id['mri_date'] - df_id['tau_date']).dt.days / 30)
#Keep only images that were taken within 3 months (MRI-TAU)
adni1 = df_id[(df_id['date_diff'] <= 3)].copy()
print(adni1.Individual.nunique())

In [ ]:
GUID_img = adni1[['Individual', 'GUID', 'mri_date','Timepoint', 'Alt ImageID']]

In [ ]:
# df_=df.copy()
# df = df_.drop(columns=['GUID', 'Alt ImageID'])
# df = df.drop_duplicates()

In [ ]:
print(adni1.Individual.nunique())
print(adni1.columns)

# Scalar data

In [ ]:
scalar = pd.read_csv(os.path.join(dir, 'ADNI_SCALAR_20250617.csv'), skip_blank_lines=True,  skipfooter=2)
scalar = scalar.rename(columns={'InputIds':'GUID', 'TimePoint':'Timepoint', 'EXAMDATE':'HMSTROKE_date', 'EXAMDATE.1':'CDGLOBAL_date', 'EXAMDATE.2':'DXPARK_date', 'EXAMDATE.3':'MMSCORE_date'})

In [ ]:
demo = scalar[['Individual', 'PTGENDER', 'PTDOBMM', 'PTDOBYY', 'PTEDUCAT']]
demo = demo.drop_duplicates()
# # moca
# moca = scalar[['Individual', 'CDGLOBAL_date', 'CDGLOBAL']].drop_duplicates()
# moca['CDGLOBAL_date'] = pd.to_datetime(moca['CDGLOBAL_date'], errors='coerce')
# # moca
# moca = scalar[['Individual', 'MMSCORE_date', 'MMSCORE']].drop_duplicates()
# moca['MMSCORE_date'] = pd.to_datetime(moca['MMSCORE_date'], errors='coerce')
# # Parkinson's
# park = scalar[['Individual', 'DXPARK_date', 'DXPARK']].drop_duplicates()
# park['DXPARK_date'] = pd.to_datetime(park['DXPARK_date'], errors='coerce')
# # Stroke
# stroke = scalar[['Individual', 'HMSTROKE_date', 'HMSTROKE']].drop_duplicates()
# stroke['HMSTROKE_date'] = pd.to_datetime(stroke['HMSTROKE_date'], errors='coerce')

## demographics

In [ ]:
# merge dataset
demo_tau = pd.merge(tau, demo, on='Individual',  how='left')
demo_tau = demo_tau.drop_duplicates()
demo_tau_clean = demo_tau.dropna()

In [ ]:
#%% Propagate immutable information
# Fill missing values within each Individual using forward and backward fill
merge1 = demo_tau.sort_values(by=['Individual']).copy()

merge1[['PTDOBMM', 'PTDOBYY','PTGENDER','PTEDUCAT', ]] = (
    merge1.groupby('Individual')[['PTDOBMM', 'PTDOBYY','PTGENDER','PTEDUCAT']]
    .transform(lambda group: group.bfill().ffill())
    .reset_index(level=0, drop=True)  # Drop extra index added by groupby
)

In [ ]:
merge1.Individual.nunique()

In [ ]:
scalar_orig =  pd.merge(scalar, orig, on=['Individual'], how='left')
print(scalar_orig.Individual.nunique())


In [ ]:
# merge data
merge0 = pd.merge(scalar, df, on=['Individual', 'Timepoint'], how='left')
merge0 = merge0.drop(columns=['GUID', 'freesurfer_6_0_0_aparc_thickness_GUID'])
merge0 = merge0.drop_duplicates()
print(merge0.Individual.nunique())

In [ ]:
#%% Propagate immutable information
# Fill missing values within each Individual using forward and backward fill
merge1 = merge0.sort_values(by=['Individual', 'Timepoint']).copy()

merge1[['PTDOBMM', 'PTDOBYY','PTGENDER','PTEDUCAT', 'HMSTROKE', 'DXPARK']] = (
    merge1.groupby('Individual')[['PTDOBMM', 'PTDOBYY','PTGENDER','PTEDUCAT','HMSTROKE', 'DXPARK']]
    .transform(lambda group: group.bfill().ffill())
    .reset_index(level=0, drop=True)  # Drop extra index added by groupby
)

merge1[['PTDOBMM', 'PTDOBYY','PTGENDER','PTEDUCAT', 'HMSTROKE', 'DXPARK']] = (
    merge1.groupby('Individual')[['PTDOBMM', 'PTDOBYY','PTGENDER','PTEDUCAT','HMSTROKE', 'DXPARK']]
    .transform(lambda group: group.bfill().ffill())
    .reset_index(level=0, drop=True)  # Drop extra index added by groupby
)

merge1[['LONIUID', 'tau_date', 'date_diff']] = (
    merge1.groupby(['Individual','Timepoint'])[['LONIUID','tau_date','date_diff']]
    .transform(lambda group: group.bfill().ffill())
    .reset_index(level=0, drop=True)  # Drop extra index added by groupby
)



In [ ]:
print(merge1.Individual.nunique())
print(merge1.LONIUID.nunique())
#print(merge1.GUID.nunique())


In [ ]:
merge1

In [ ]:
adni2 = merge1.copy()#.dropna(subset='LONIUID')

In [ ]:
adni2

In [ ]:
orig = pd.read_csv(os.path.join(dir, 'adni_original/PTDEMOG_05Jun2025.csv'), 
                   usecols=['PTID','RID', 'VISDATE', 'PTGENDER', 'PTDOB', 'PTEDUCAT'])
orig = orig.rename(columns={'PTID':'Individual', 'PTDOB':'DOB'})
orig = orig.drop(columns=['VISDATE','RID'])

tmp = orig[orig['Individual'].isin(adni2['Individual'])]
tmp['DOB'] = pd.to_datetime(tmp['DOB'])

In [ ]:
adni2

In [ ]:
# Step 1: Drop rows with missing month or year of birth
print(adni2.Individual.nunique())
print(adni2.LONIUID.nunique())

#change to integer
adni2[['PTDOBMM', 'PTDOBYY']] = adni2[['PTDOBMM', 'PTDOBYY']].astype('Int64')

# put MM and YY of birth together and change to datetime format
adni2['DOB'] = adni2['PTDOBYY'].astype(str) + adni2['PTDOBMM'].astype(str)
adni2['DOB'] = pd.to_datetime(adni2['DOB'], format='%Y%m', errors='coerce')


In [ ]:
dob_lookup = tmp.set_index('Individual')['DOB']
tmp = pd.merge(adni2, dob_lookup, on='Individual')

tmp['DOB'] = np.where(pd.notna(tmp['DOB_y']),
                             tmp['DOB_y'],
                             tmp['DOB_x'])

print(tmp.Individual.nunique())
adni3 = tmp.drop(columns=['DOB_x','DOB_y'])

In [ ]:
adni3['tau_date'] = pd.to_datetime(adni3['tau_date'])
adni3['mri_date'] = pd.to_datetime(adni3['mri_date'])

# Calculate age at FDG scan
adni3['age_TAU'] = adni3['tau_date'] - adni3['DOB']
adni3['age_TAU'] = adni3['age_TAU']/pd.Timedelta('365.25 days')
adni3['age_TAU'] = adni3['age_TAU'].round(2)
# Calculate age at MRI scan
adni3['age_MRI'] = adni3['mri_date'] - adni3['DOB']
adni3['age_MRI'] = adni3['age_MRI']/pd.Timedelta('365.25 days')
adni3['age_MRI'] = adni3['age_MRI'].round(2)


In [ ]:
adni3.Individual.nunique()

In [ ]:
fs_keys = adni3[['freesurfer_6_0_0_aparc_thickness_GUID']]
tau_keys = adni3[['LONIUID']]
tau_fs = adni1.drop(columns=['RID', 'VISCODE', 'SITEID', 'Individual', 'tau_date', 'PROCESSDATE', 'TRACER', 'TRACER_SUVR_WARNING'])

## Organize MOCA
#https://www.smchealth.org/sites/main/files/file-attachments/moca-instructions-english_2010.pdf
#https://www.smchealth.org/sites/main/files/file-attachments/moca-instructions-english_2010.pdf

In [ ]:
moca_df = pd.read_csv(os.path.join(dir, 'adni_moca_20250612.csv'))
moca_df = moca_df[moca_df['Phase'].notna()]
#ADjust variables
moca_df = moca_df.drop(columns=['IMMT1W1', 'IMMT1W2', 'IMMT1W3', 'IMMT1W4', 'IMMT1W5', 'IMMT2W1', 'IMMT2W2', 'IMMT2W3', 'IMMT2W4', 'IMMT2W5' ])

# Letters = 0 or 1 error gives 1 point, else is 0
moca_df['letters_bi'] = np.where(moca_df['LETTERS'].isin([0, 1]), 1, 0)
moca_df = moca_df.drop(columns=['LETTERS'])

#FLUENCY
# if more than 0 < 11 < 1
moca_df['fluency'] = (moca_df['FFLUENCY'] >= 11).astype(int)
moca_df = moca_df.drop(columns=['FFLUENCY'])

# SERIAL 
#Give no (0) points for no correct subtractions, 1 point for one correction subtraction, 2 points for two-to-three correct subtractions, and 3 points if the participant successfully makes four or five correct subtractions
serial_cols = [col for col in moca_df.columns if col.startswith('SERIAL')]
moca_df['serial_sum'] = moca_df[serial_cols].sum(axis=1)
moca_df['serial_sum'] = np.where(moca_df['serial_sum'].isin([4, 5]), 3, 0)
moca_df = moca_df.drop(columns=['SERIAL1', 'SERIAL2', 'SERIAL3', 'SERIAL4', 'SERIAL5'])

#DEL
# 1=Correct with No Cue; 2=Correct with Category Cue; 3=Correct with Mult. Choice Cue; 0=Incorrect
# 1 point only with correct with no cue
moca_df['delw1'] = (moca_df['DELW1'] == 1).astype(int)
moca_df['delw2'] = (moca_df['DELW2'] == 1).astype(int)
moca_df['delw3'] = (moca_df['DELW3'] == 1).astype(int)
moca_df['delw4'] = (moca_df['DELW4'] == 1).astype(int)
moca_df['delw5'] = (moca_df['DELW5'] == 1).astype(int)
moca_df = moca_df.drop(columns=['DELW1', 'DELW2', 'DELW3', 'DELW4', 'DELW5'])

moca_df['moca_score'] = moca_df.iloc[:,8:].sum(axis=1)


In [ ]:
moca_df

In [ ]:
# put moca together with the data 
moca_df_short = moca_df[['InputIds','moca_score']]
moca_df_short = moca_df_short.rename(columns={'InputIds':'GUID'})
adni4 = pd.merge(adni3, moca_df_short, on= ['GUID'], how='left')

In [ ]:
# put moca together with the data 
# moca_df_short = moca_df[['freesurfer_6_0_0_aparc_thickness_GUID', 'moca_score']]
# adni4 = pd.merge(adni3, moca_df_short, on= 'freesurfer_6_0_0_aparc_thickness_GUID', how='left')

In [ ]:
adni4[['moca_score']] = (
    adni4.groupby(['Individual','Timepoint'])[['moca_score']]
    .transform(lambda group: group.bfill().ffill())
    .reset_index(level=0, drop=True)  # Drop extra index added by groupby
)

In [ ]:
moca_df

In [ ]:
adni4.Individual.nunique()

## Merge ADNI with biological data
Amyloid SUVR
ptau181 : https://pubmed.ncbi.nlm.nih.gov/32333900/
          https://pubmed.ncbi.nlm.nih.gov/29626426/
          https://adni.bitbucket.io/reference/docs/UGOTPTAU181/UGOT_Lab_-_ADNI_1-Go-2_-_Method_-_Plasma_P-tau181_longitudinal.pdf
Alpha-synuclein

In [ ]:
#Load
adni_bio = pd.read_csv(os.path.join(dir, 'ADNI_BIOMARKERS_SUV_TAU_20250613.csv'))
# Be sure dates are in the right format
adni_bio['EXAMDATE_asyn'] = pd.to_datetime(adni_bio['EXAMDATE'], errors='coerce')
adni_bio['EXAMDATE_SUVR1.11'] = pd.to_datetime(adni_bio['EXAMDATE.1'], errors='coerce')
adni_bio['EXAMDATE_SUVR1.08'] = pd.to_datetime(adni_bio['EXAMDATE.2'], errors='coerce')
adni_bio['EXAMDATE_ptau181'] = pd.to_datetime(adni_bio['EXAMDATE.3'], errors='coerce')
# Exclude the examdate in wrong format
adni_bio = adni_bio.drop(columns=['EXAMDATE', 'EXAMDATE.1','EXAMDATE.2','EXAMDATE.3', 'registry_vdate'])
# rename columns
adni_bio = adni_bio.rename(columns={'Result': 'alpha_syn'})

In [ ]:
# Create 'amy_status' using the first non-null value from the two columns
adni_bio['amy_status'] = np.where(pd.notna(adni_bio['SUMMARYSUVR_WHOLECEREBNORM_1.11CUTOFF']),
                                           adni_bio['SUMMARYSUVR_WHOLECEREBNORM_1.11CUTOFF'],
                                           adni_bio['SUMMARYSUVR_WHOLECEREBNORM_1.08CUTOFF'])


In [ ]:
adni_bio_short = adni_bio[['freesurfer_6_0_0_aparc_thickness_GUID', 
                           'amy_status', 'PLASMAPTAU181','alpha_syn']]

adni5 = pd.merge(adni4, adni_bio_short, on= 'freesurfer_6_0_0_aparc_thickness_GUID', how='left')

adni5[['amy_status']] = (
    adni5.groupby(['Individual','Timepoint'])[['amy_status']]
    .transform(lambda group: group.bfill().ffill())
    .reset_index(level=0, drop=True)  # Drop extra index added by groupby
)

adni5_ = adni5.dropna(subset='amy_status')

In [ ]:
print(adni4.Individual.nunique())
print(adni5.Individual.nunique())

# Diagnosis


In [ ]:
adni_diag = pd.read_csv(os.path.join(dir, 'ADNI_DIAGNOSIS_20250616.csv'))
adni_diag

In [ ]:
# Load
adni_diag = pd.read_csv(os.path.join(dir, 'ADNI_DIAGNOSIS_20250616.csv'))

#change missing to na, and None to na
adni_diag = adni_diag.replace('None', np.nan)
adni_diag = adni_diag.replace(-4, np.nan)
adni_diag = adni_diag.replace('-4', np.nan)

#Diagnosis status

# varibale 'diag_current' is based on DIAGNOSIS and DXCURREN variables that should be the same
adni_diag['diag_current'] = np.where(pd.notna(adni_diag['DIAGNOSIS']),
                             adni_diag['DIAGNOSIS'],
                             adni_diag['DXCURREN'])

# variable diag_change is based on diag_current and DXCHANGE
adni_diag['diag_change'] = np.where(pd.notna(adni_diag['diag_current']),
                             adni_diag['diag_current'],
                             adni_diag['DXCHANGE'])

adni_diag['diagnosis_CD_raw'] = adni_diag['diag_change']

- Dictionary
- DXCHANGE:  Which best describes the participant's change in cognitive status from last visit to current visit:
 1=Stable: NL to NL; 2=Stable: MCI to MCI; 3=Stable: Dementia to Dementia; 4=Conversion: NL to MCI; 5=Conversion: MCI to Dementia; 6=Conversion: NL to Dementia; 7=Reversion: MCI to NL; 8=Reversion: Dementia to MCI; 9=Reversion: Dementia to NL
- DXCURREN:
 1=NL;2=MCI;3=AD
- DXCONV: Has there been a conversion or reversion to NL/MCI?
 1=Yes - Conversion;2=Yes - Reversion; 0=No
- DXREV:  If YES - REVERSION, choose type
 1=MCI to Normal Control; 2=AD to MCI; 3=AD to Normal Control
- DXNORM: Normal
 1=Yes
- DXMCI: 
 1=Yes
- DXMDES: If Mild Cognitive Impairment, select any that apply:
 1=MCI - Memory features (amnestic); 2=MCI - Non-memory features (non-amnestic)
 1=MCI (Memory features); 2=MCI (Non-memory features)
- DXMPTR1: If MCI - Memory features, complete the following (Petersen Criteria, see procedures manual for details): i. Subjective memory complaint
 1=Yes; 0=No
- DXMDUE: suspected cause of MCI
 1=MCI due to Alzheimer's Disease; 2=MCI due to other etiology
- DXMOTHET: If MCI due to other etiology, select box(es) to indicate reason:
 1=Fronto-temporal Dementia; 2=Parkinson's Disease; 3=Huntington's Disease; 4=Progressive Supranuclear Palsy; 8=Corticobasal Degeneration; 9=Vascular Dementia; 10=Prion-Associated Dementia; 14=Other (specify)
- DXDDUE:  3b. Suspected cause of dementia
 1=Dementia due to Alzheimer's Disease; 2=Dementia due to other etiology
- DXAD: Alzheimer's disease
 1=Yes
- DXAPP: If Dementia due to Alzheimer's Disease, indicate likelihood:
 1=Probable; 2=Possible
- DXAPOSS: If Possible AD, select box(es) to indicate reason:
 1=Atypical clinical course or features (specify); 2=Stroke(s); 3=Depression; 4=Delirium; 5=Parkinsonism; 6=Metabolic / Toxic Disorder (specify); 7=Other (specify)
- DXPARK:  4b. Parkinsonism symptoms present?
 1=Yes; 0=No
- DXODES:  If Other Dementia, select box which indicates best diagnosis:
 1=Frontal; 3=Huntington's Disease; 5=Alcohol-related Dementia; 6=NPH; 7=Major Depression; 9=Vascular Dementia; 10=Prion-Associated Dementia; 11=HIV; 12=Primary Progressive Aphasia; 13=Posterior Cortical Dysfunction; 14=Other (specify)
- DIAGNOSIS: 1. Which best describes the participant's current diagnosis?
 1=CN; 2=MCI; 3=Dementia
- BCSTROKE: 10. Did subject have a stroke? ( Diagnostic Summary - Baseline Changes)
 1=Yes; 0=No

In [ ]:
mytable = TableOne(adni_diag, columns=['diagnosis_CD_raw', 'diag_current', 'diag_change',
       'DXCHANGE', 'DXCURREN', 'DXCONV', 'DXREV', 'DXNORM', 'DXMCI', 'DXMDES', 'DXMPTR1',
       'DXMDUE', 'DXMOTHET', 'DXDDUE', 'DXAD', 'DXAPP', 'DXAPOSS', 'DXPARK',
       'DXODES', 'DIAGNOSIS', 'BCSTROKE'])
print(mytable.tabulate(tablefmt = "fancy_grid"))


In [ ]:
#%% Create new diagnosis (only 1, 2, 3) and calculate conversion time from baseline
# 1=Stable: NL to NL; 2=Stable: MCI to MCI; 3=Stable: Dementia to Dementia; 
# 4=Conversion: NL to MCI; 5=Conversion: MCI to Dementia; 6=Conversion: NL to Dementia; 
# 7=Reversion: MCI to NL; 8=Reversion: Dementia to MCI; 9=Reversion: Dementia to NL	
# Define mapping dictionary
diagnosis_mapping = {
    4: 2,
    5: 3,  
    6: 3,
    7: 1,
    8: 2,
    9: 1
}

# Apply mapping to the column
#remove None and convert to numeric to map properly
adni_diag['diagnosis_CD_raw'] = adni_diag['diagnosis_CD_raw'].replace('None', np.nan)
adni_diag['diagnosis_CD'] = adni_diag['diagnosis_CD_raw'].astype(float).replace(diagnosis_mapping)

#adni_diag['diagnosis_CD'] = adni_diag['diagnosis_CD'].replace('None', np.nan)
#adni_diag['diagnosis_CD'] = adni_diag['diagnosis_CD'].astype(float)
adni_diag = adni_diag.dropna(subset=['diagnosis_CD'])

#change to datetime
adni_diag['EXAMDATE'] = pd.to_datetime(adni_diag['EXAMDATE'])
# Sort by Individual and EXAMDATE to ensure proper chronological order
adni_diag = adni_diag.sort_values(by=['Individual', 'EXAMDATE'])
#Calculate time between tp
adni_diag['time_from_bl'] = adni_diag.groupby('Individual')['EXAMDATE'].transform(lambda x: (x - x.min()).dt.days / 365.25)

# Compute diagnosis change
adni_diag['diagnosis_changed'] = adni_diag.groupby('Individual')['diagnosis_CD'].transform(
    lambda x: x != x.shift()
)
# First record for each Individual: force to False
adni_diag.loc[adni_diag.groupby('Individual').head(1).index, 'diagnosis_changed'] = False

# Flag Ids that has changed before two years
adni_diag['exclude_ID'] = (
    (adni_diag['diagnosis_changed']) & 
    (adni_diag['time_from_bl'] <= 1)
)


In [ ]:
adni_diag['cn_to_mci'] = (
    (adni_diag['diagnosis_CD'].shift() == 1) &   # Previous was MCI
    (adni_diag['diagnosis_CD'] == 2) &           # Current is AD
    (adni_diag['diagnosis_changed']) &           # Must be a real change
    (adni_diag['time_from_bl'] > 0)              # Not baseline
)


In [ ]:
adni_diag['mci_to_ad'] = (
    (adni_diag['diagnosis_CD'].shift() == 2) &   # Previous was MCI
    (adni_diag['diagnosis_CD'] == 3) &           # Current is AD
    (adni_diag['diagnosis_changed']) &           # Must be a real change
    (adni_diag['time_from_bl'] > 0)              # Not baseline
)


In [ ]:
mytable = TableOne(adni_diag, columns=['diagnosis_CD', 'diagnosis_changed', 'exclude_ID', 'cn_to_mci', 'mci_to_ad', 'diagnosis_CD_raw'])
print(mytable.tabulate(tablefmt = "fancy_grid"))


In [ ]:
adni_diag[['diagnosis_CD', 'diagnosis_CD_raw']] = (
    adni_diag.groupby(['Individual','TimePoint'])[['diagnosis_CD', 'diagnosis_CD_raw']]
    .transform(lambda group: group.bfill().ffill())
    .reset_index(level=0, drop=True)  # Drop extra index added by groupby
)

In [ ]:
adni_diag_short = adni_diag.drop(columns=['Project', 'Individual', 'AltId', 'TimePoint', 'InputIds','DXPARK', 'DXCHANGE',
       'DXCURREN', 'DXCONV', 'DXREV', 'DXNORM', 'DXMCI', 'DXMDES', 'DXMPTR1',
       'DXMDUE', 'DXMOTHET', 'DXDDUE', 'DXAD', 'DXAPP', 'DXAPOSS', 'DXODES',
       'DIAGNOSIS', 'BCSTROKE'])



In [ ]:
adni6 = pd.merge(adni5, adni_diag_short, on= 'freesurfer_6_0_0_aparc_thickness_GUID', how='left')
adni6_ = adni6.dropna(subset='diagnosis_CD')

In [ ]:
adni6.Individual.nunique()

# APOE

In [ ]:
adni_apoe = pd.read_csv(os.path.join(dir, 'ADNI_APOE_20250616.csv'))

adni_apoe['APGEN1'] = adni_apoe['APGEN1'].astype(float)
adni_apoe['APGEN2'] = adni_apoe['APGEN2'].astype(float)

adni_apoe['apoe_e4'] = np.where(
    (adni_apoe['APGEN1'] == 4) | (adni_apoe['APGEN2'] == 4),
    1,  # True case
    0   # False case
)

# Count number of APOE ε4 alleles (each allele == 4)
adni_apoe['apoe_e4_count'] = (
    (adni_apoe['APGEN1'] == 4).astype(int) + 
    (adni_apoe['APGEN2'] == 4).astype(int)
)


adni_apoe[['apoe_e4', 'apoe_e4_count']] = (
    adni_apoe.groupby(['Individual','TimePoint'])[['apoe_e4', 'apoe_e4_count']]
    .transform(lambda group: group.bfill().ffill())
    .reset_index(level=0, drop=True)  # Drop extra index added by groupby
)

adni_apoe_short = adni_apoe[['freesurfer_6_0_0_aparc_thickness_GUID', 'apoe_e4', 'apoe_e4_count']]
adni7 = pd.merge(adni6, adni_apoe_short, on= 'freesurfer_6_0_0_aparc_thickness_GUID', how='left')

In [ ]:
adni7.Individual.nunique()

# Merge TAU fs output

In [ ]:
adni_tau_fs = pd.merge(tau_fs, adni7, on='LONIUID', how='left')

In [ ]:
adni_tau_fs = adni_tau_fs.drop(columns=['HMSTROKE', 'DXPARK', 'PTDOBMM', 'PTDOBYY', 'diag_current', 'diag_change'])

In [ ]:
adni_tau_fs.Individual.nunique()

In [ ]:
# Ensure dates are sorted so "first" means earliest
adni_tau_fs = adni_tau_fs.sort_values(by=['Individual', 'tau_date'])
adni_tau_fs = adni_tau_fs.dropna(subset=['freesurfer_6_0_0_aparc_thickness_GUID']) 

# Keep only the first tau_date per Individual
tmp = adni_tau_fs.drop_duplicates(subset='Individual', keep='first')


In [ ]:
tmp.Individual.nunique()

In [ ]:
tmp

In [ ]:
adni_tau_fs.columns[320:370]

# TABLE 1 

In [ ]:
#table_df = adni_tau_fs.copy()
table_df = adni7.copy()

# Cleaning of data

table_df['CDGLOBAL'] = table_df['CDGLOBAL'].replace(-1, np.nan)
#table_df['PLASMAPTAU181'] = table_df['PLASMAPTAU181'].replace('None', np.nan)
# Alpha synuclein
# https://alz-journals.onlinelibrary.wiley.com/doi/full/10.1002/alz.14571
# "CSF samples were classified into one of four categories: “PD/DLB-like Detected” (Type 1) if α-syn aggregates were consistent with seeds observed in Parkinson's disease and dementia with Lewy bodies (DLB); 
# “MSA-like Detected” (Type 2) if α-syn aggregates matched seeds that are typically seen in multiple system atrophy; “Not Detected” if no α-syn aggregates were observed; or “Indeterminate” if samples did not 
# yield a definite result after two tests. For all subsequent analyses in this study, only Type 1 cases (n = 196; 34 CU and 162 CI) were considered SAA+, and only “Not Detected” cases (n = 633; 147 CU and 486 CI) 
# were considered SAA–. Both Type 2 (n = 2; 1 CU and 1 CI) and Indeterminate cases (n = 7; 1 CU and 6 CI) were excluded. All CSF α-syn SAA analyses were performed with analysts blinded to participants’ 
# demographic details, clinical profiles, and AD biomarker data."

table_df['alpha_syn_'] = table_df['alpha_syn'].replace('Detected-1', '1')
table_df['alpha_syn_'] = table_df['alpha_syn'].replace('Detected-2', '1')
table_df['alpha_syn_'] = table_df['alpha_syn'].replace('Not_Detected', '0')
table_df['alpha_syn_'] = table_df['alpha_syn'].replace('Indeterminate', '2')
table_df['alpha_syn_'] = table_df['alpha_syn'].replace('None', '0')


In [ ]:
# Organize groups
#make sure data is numeric
table_df['diagnosis_CD'] = table_df['diagnosis_CD'].astype('float')  
table_df['amy_status'] = table_df['amy_status'].astype('float')

# Create the conditions and code for groups
conditions = [
    (table_df['diagnosis_CD'] == 1) & (table_df['amy_status'] == 0),  # CN, amy- 0
    (table_df['diagnosis_CD'] == 1) & (table_df['amy_status'] == 1),  # CN, amy+ 1
    (table_df['diagnosis_CD'] == 2) & (table_df['amy_status'] == 0),  # MCI, amy- 2
    (table_df['diagnosis_CD'] == 2) & (table_df['amy_status'] == 1),  # MCI, amy+ 3
    (table_df['diagnosis_CD'] == 3) & (table_df['amy_status'] == 0),  # AD, amy- 4
    (table_df['diagnosis_CD'] == 3) & (table_df['amy_status'] == 1),  # AD, amy+ 5
]

group_codes = [0, 1, 4, 2, 5, 3]

table_df['group_code'] = np.select(conditions, group_codes, default=np.nan)

group_labels = {
    0: 'CN amy-',
    1: 'CN amy+',
    4: 'MCI amy-',
    2: 'MCI amy+',
    5: 'AD amy-',
    3: 'AD amy+'
}

table_df['group_label'] = table_df['group_code'].map(group_labels)

table_df = table_df[~table_df['group_code'].isin([4, 5])]

In [ ]:
a = table_df[['Individual','amy_status','diagnosis_CD']].drop_duplicates()

In [ ]:
table_df1 = table_df.dropna(subset='group_code')
columns = ['age_MRI', 'age_TAU', 'amy_status','group_label', 
           'MMSCORE','CDGLOBAL', 'moca_score', 
           'PLASMAPTAU181', 'alpha_syn', 'time_from_bl', 'diagnosis_changed', 
           'cn_to_mci', 'mci_to_ad'
        ]

categorical = ['CDGLOBAL', 'alpha_syn', 'cn_to_mci', 'mci_to_ad', 'amy_status']
continuous = ['age_MRI', 'age_TAU', 'MMSCORE', 'moca_score' ]
nonnormal = []
rename={'age_MRI': 'Age at MRI scan', 'age_TAU': 'Age at FDG scan', 
        'MMSCORE':'moca','CDGLOBAL':'moca-SOB', 'moca_score':'MoCA', 
        'PLASMAPTAU181':'Plasma_pTau181', 'alpha_syn':'Aplha-synuclein'}
groupby='group_code'

#https://tableone.readthedocs.io/en/latest/index.html
mytable = TableOne(table_df1, columns=columns, 
                    categorical=categorical, continuous=continuous, 
                    groupby=groupby, 
                    nonnormal=nonnormal, 
                    rename=rename, pval=True)

print(mytable.tabulate(tablefmt="github"))
#“github”, “grid”, “fancy_grid”, “rst”, “html”, and “latex”.

In [ ]:
table_df1.to_csv(os.path.join(dir, 'ADNI_PET_TAU_DATA_FS_20250617.csv'))

In [ ]:
#some visual stats
import seaborn as sns

sns.boxplot(data=table_df1, x='group_code', y='moca_score', hue='group_code')
#sns.regplot(data=table_df, x='group_code', y='CDGLOBAL', scatter=False, color='r')